In [1]:
from manim import *

Manim Community v0.16.0.post0

In [373]:
FINAL_EQN_VGROUP = None

In [512]:
EQN_SIZE = 70
EQN_SIZE_2 = 60

K_COLOR = BLUE
X_COLOR = GREEN
TEXT_COLOR = GRAY_B
DELAY = 0.1
SKIP_MODE = False #mode to speed up animatins

def printEnumerate(thing):
    for a,b in enumerate(thing):
        print(a,b)

def MathTexAndColor(*tex):
        eqn = MathTex(*tex,  font_size = EQN_SIZE)
        eqn.set_color_by_tex("k", K_COLOR)
        eqn.set_color_by_tex("x", X_COLOR)
        return eqn


    
def set_new_location(A,B):
    A.set_x(B.get_x())
    A.set_y(B.get_y())
    
def color_and_move_this_eqn(eqn_name,eqn_target):
    eqn_name.set_color_by_tex("k", K_COLOR)
    eqn_name.set_color_by_tex("x", X_COLOR)
    eqn_name.next_to(eqn_target,DOWN*8)
    eqn_name.align_to(eqn_target,LEFT)

In [513]:
%%manim -ql -v CRITICAL gaussian_moments

#IMPORTANT: -ql = low quality. Remove flag before export!


class gaussian_moments(Scene):
    
    def skip_create(self,*args):
        if SKIP_MODE == False:
            self.play(Create(*args))
            self.wait(DELAY)
        else:
            self.add(*args)
    
    def construct(self):
        
        #setup the equation but in parts so we can manipualte it
        eqn_start = MathTex(r"\mathbb{E}[Z^{{k}}]",r"=", r"\frac{1}{\sqrt{2\pi}}", r" \intop_{-\infty}^{\infty}", font_size=EQN_SIZE)
        eqn_start.set_color_by_tex("k", K_COLOR)
        
        #print("eqn_start")
        #for i,x in enumerate(eqn_start):
        #    print(i,x)
        
        eqn_midA = MathTexAndColor(r" {{x}}^{{k}} \Bigr) ")
        eqn_midA[-1].color = BLACK #make the bracket black
        eqn_midB = MathTexAndColor(r" e^{-\frac{1}{2}{{x}}^2}} {{\Bigr)}}")
        eqn_midB[-1].color = BLACK #make the bracket black
        eqn_end = MathTexAndColor(r"d {{x}}")
        
        eqn = VGroup(eqn_start,eqn_midA,eqn_midB,eqn_end).arrange(RIGHT)
        eqn.shift(2.25*UP+0.5*LEFT)
        #eqn.to_corner(UL)
        
        #Now make a copy of the equation, but spaced so there is enough room to covert x^k into x^{k-1} x
        eqn_midA_final = MathTexAndColor(r" {{x}}^{ {{k}} - 1 } {{\Bigl(}} {{x}}  ")
        eqn_midA_final[5].color = BLACK #make the bracket black
        
        #arrange eqn using x^{k-1}x
        eqn_w_space = VGroup(eqn_start.copy(),eqn_midA_final,eqn_midB.copy(),eqn_end.copy()).arrange(RIGHT)
        eqn_w_space.align_to(eqn,UP)
        eqn_w_space.align_to(eqn,LEFT)
        eqn_w_space.shift(0.5*LEFT)
        #eqn_w_space.to_corner(UL)
        
        #replace x^{k-1}x by x^k (so everything else will be spaced correctly)
        eqn_midA_space = eqn_midA.copy() 
        set_new_location(eqn_midA_space,eqn_midA_final) 
        eqn_w_space[1] = eqn_midA_space
        
        #eqn2.set_color_by_tex("k", K_COLOR)
        #eqn2.set_color_by_tex("x", X_COLOR)
        
        #####
        #Animate the equation in slowly piece by piece so voice over explanation can be added
        #####
        
        self.play(FadeIn(eqn_start[0:4])) #fade in first part
        self.wait(DELAY)
        
        self.play(FadeIn(eqn_start[5],eqn_end)) #fade in integral sign
        self.wait(DELAY)
        
        pdf_text = Text("Probability Density Function")
        pdf_text.color=TEXT_COLOR
        pdf_text.next_to(eqn_start[5],DOWN)
        pdf_text.set_x(0)
        
        pdf = MathTex(r"p({{x}})=\frac{1}{\sqrt{2\pi}} e^{-\frac{1}{2}{{x}}^2}}", color=TEXT_COLOR, font_size=EQN_SIZE_2 )
        pdf.set_color_by_tex("x",X_COLOR)
        pdf.next_to(pdf_text,DOWN)
        
        pdf_graph = FunctionGraph(lambda t: np.exp(-0.5*t**2),color=TEXT_COLOR)
        pdf_graph.next_to(pdf,DOWN)
        
        pdf_list = [pdf,pdf_graph]
        self.play(Create(pdf_text), *[FadeIn(mob) for mob in pdf_list]) #fade in PDF
        self.wait(DELAY)
        self.play(FadeIn(eqn_start[4],eqn_midB)) #fade in 1/sqrt(2pi) and e^-x^2/2
        self.wait(DELAY)
        pdf_list = [pdf,pdf_text,pdf_graph]
        self.play(*[FadeOut(mob) for mob in pdf_list]) #fade out PDF
        self.wait(DELAY)

        box = SurroundingRectangle(eqn_start[1]) #box to surround z^k
        self.play(FadeIn(eqn_midA),Create(box))
        self.wait(DELAY)
        self.play(FadeOut(box))
        self.wait(DELAY)
        
        #if SKIP_MODE == False: 
        #    for parts in [ [eqn_start[0:4]],[eqn_start[5],eqn_end],[eqn_start[4],eqn_midB],[eqn_midA]]:
        #        self.play(FadeIn(*parts))
        #        self.wait(DELAY)
        #else:
        #    self.add(eqn)
            
        if SKIP_MODE == False:
            int_by_parts_q = Text("Integration by parts?")
            int_by_parts_q.color = TEXT_COLOR
            int_by_parts_q.next_to(eqn_midA,UP)
            int_by_parts_q.align_to(eqn_midA,LEFT)
            self.play(Create(int_by_parts_q))
            self.wait(DELAY)
            self.play(FadeOut(int_by_parts_q))
            self.wait(DELAY)
            
            
            
        self.play(ReplacementTransform(eqn,eqn_w_space))
        self.play(TransformMatchingTex(eqn_midA_space,eqn_midA_final))
        self.wait(DELAY)
        self.play(FadeToColor(eqn_midB[-1], color=WHITE),FadeToColor(eqn_midA_final[5], color=WHITE))
        self.wait(DELAY)
        
        back_brace = BraceLabel(VGroup(eqn_midA_final[-4:-1],eqn_midB),r"\text{Do }\int")
        back_brace.color = TEXT_COLOR
        self.skip_create(back_brace)
        
        front_brace = BraceLabel(eqn_midA_final[0:5],r"\text{Do }\frac{d}{dx}")
        front_brace.align_to(back_brace,UP)
        front_brace.color = TEXT_COLOR
        self.skip_create(front_brace)
        
        int_by_parts = Text("Integration by parts!")
        int_by_parts.color = TEXT_COLOR
        int_by_parts.next_to(eqn_midA_final,UP)
        int_by_parts.align_to(eqn_midA_final,LEFT)
        self.skip_create(int_by_parts)
        
        
        eqn_ibp = MathTex(r"=", r"\frac{-1}{\sqrt{2\pi}}", r" \intop_{-\infty}^{\infty}", r"({{k}}-1){{x}}^{{{k}}-2}",r"\bigl(-e^{-\frac{1}{2}{{x}}^2}}\bigr)",r"d{{x}}", font_size=EQN_SIZE)
        
        
        eqn_ibp_no_minus = MathTex(r"=", r"\frac{1}{\sqrt{2\pi}}", r"\intop_{-\infty}^{\infty}", r"({{k}}-1){{x}}^{{{k}}-2}",r"\bigl(e^{-\frac{1}{2}{{x}}^2}}\bigr)",r"d{{x}}", font_size=EQN_SIZE)
        
        
        eqn_ibp_k = MathTex(r"=", r"({{k}}-1)", r"\frac{1}{\sqrt{2\pi}}", r"\intop_{-\infty}^{\infty}", r"{{x}}^{{{k}}-2}",r"\bigl(e^{-\frac{1}{2}{{x}}^2}}\bigr)",r"d{{x}}", font_size=EQN_SIZE)
        
        #line up to the equals sign from the first eqn
        color_and_move_this_eqn(eqn_ibp,eqn_start[3])
        color_and_move_this_eqn(eqn_ibp_no_minus,eqn_start[3])
        color_and_move_this_eqn(eqn_ibp_k,eqn_start[3])
        
        
        
        
        #print("eqn_midA_final")
        #for e,i in enumerate(eqn_midA_final):
        #    print(e,i)
            
        #print("eqn_midB")
        #for e,i in enumerate(eqn_midB):
        #    print(e,i)
            
        #print("eqn_ibp_k")
        #for e,i in enumerate(eqn_ibp_k):
        #    print(e,i)
        #    
        #print("eqn_ibp_no_minus")
        #for e,i in enumerate(eqn_ibp_no_minus):
        #    print(e,i)
            
        
        #move over integral and 1/sqrt{2pi}
        pairs_list = [ [eqn_start[3],eqn_ibp[0]],[eqn_start[4],eqn_ibp[1]],[eqn_start[5],eqn_ibp[2]],[eqn_end,eqn_ibp[-2:]] ] 
        self.play(*[TransformFromCopy(a,b) for a,b in pairs_list])
        
        #do the derivative
        pairs_list = [ [eqn_midA_final[0:-4], eqn_ibp[3:10]] ]
        self.play(*[TransformFromCopy(a,b) for a,b in pairs_list], Indicate(front_brace))
        self.play(FadeOut(front_brace))
        self.wait(DELAY)
       
        
        #do the integral
        pairs_list = [ [eqn_midA_final[5:], eqn_ibp[10]],[eqn_midB, eqn_ibp[11:13]] ]
        self.play(*[TransformFromCopy(a,b) for a,b in pairs_list], Indicate(back_brace))
        self.play(FadeOut(back_brace),FadeOut(int_by_parts))
        self.wait(DELAY)
       
        
       
        
        #eqn_ibp_2MathTex(r"=", r"\frac{1}{\sqrt{2\pi}}", r" \intop_{-\infty}^{\infty}", r"({{k}}-1){{x}}^{{{k}}-2}",r"\bigl( e^{-\frac{1}{2}{{x}}^2}} \bigr)",r"d {{x}}", font_size=EQN_SIZE)
        
        
        self.play(ReplacementTransform(eqn_ibp,eqn_ibp_no_minus))
        self.wait(DELAY)
        
        self.play(TransformMatchingTex(eqn_ibp_no_minus,eqn_ibp_k))
        self.wait(DELAY)
        
        box = SurroundingRectangle(eqn_ibp_k[4:])
        self.skip_create(box)
        
        moment = MathTexAndColor(r"\mathbb{E}[Z^{ {{k}}-2} ]")
        moment.next_to(eqn_ibp_k[3],RIGHT)
        moment.align_to(eqn_ibp_k[3],DOWN)
        
        
        
        self.play(ReplacementTransform(eqn_ibp_k[4:],moment),FadeOut(box))
        self.wait(DELAY)
        
        RHS = VGroup(eqn_ibp_k[1],eqn_ibp_k[2],eqn_ibp_k[3],moment)
        RHS.generate_target()
        RHS.target.next_to(eqn_w_space[0][0:4],RIGHT)
        
        
        #eqn_w_space[1]
        fade_list = [eqn_ibp_k[0],eqn_w_space[0][4:],eqn[2],eqn_start[0],eqn_midA_final, eqn_w_space[2],eqn_w_space[3], eqn_end]    
        self.play(MoveToTarget(RHS), *[FadeOut(mob) for mob in fade_list] )
        self.wait(DELAY)
        
        global FINAL_EQN_VGROUP
        FINAL_EQN_VGROUP = VGroup(eqn_w_space[0][0:4],RHS) #save the final equation to be used in the next scene
        printEnumerate(FINAL_EQN_VGROUP)
        
        
        self.wait(2)

0 VGroup of 4 submobjects
1 VGroup of 4 submobjects


In [591]:
%%manim -ql -v CRITICAL double_factorials

#Note that TOP_LEFT_X and TOP_LEFT_Y are imported from the previous scene


EQN_SIZE_3 = 60
def MathTexAndColor3(*tex):
        eqn = MathTex(*tex,  font_size = EQN_SIZE_3)
        eqn.set_color_by_tex("k", K_COLOR)
        eqn.set_color_by_tex("x", X_COLOR)
        return eqn

class double_factorials(Scene):
    
    def skip_create(self,*args):
        if SKIP_MODE == False:
            self.play(Create(*args))
            self.wait(DELAY)
        else:
            self.add(*args)
    
    def construct(self):
        self.add(FINAL_EQN_VGROUP) #the final equation from the previous scene
        self.wait(DELAY)
        
        #k_Es = the list of Tex expression E[Z^k]
        M=[1,0,1,0,3,0,15,0,105]
        k_Es = [MathTexAndColor3(r"\mathbb{E}",r"[",r"Z^",str(k),"]") for k in range(9)]
        k_Es_ans = [MathTexAndColor3(str(M[k])) for k in range(9)]
        #r"\mathbb{E}",r"[",r"Z^",str(k),"]","=",
    
        #color the "k"s correctly (note that the ks are 0,1,2,3...)
        for mob in k_Es:
            mob[3].color = K_COLOR
        
        
        #add them to a group, arrange and set to side of screen
        k_Es_group = VGroup(*k_Es).arrange(RIGHT, buff = MED_LARGE_BUFF)
        k_Es_group.to_edge(LEFT)
        k_Es_group.shift(0.75*DOWN)
        
        for i in range(9):
            set_new_location(k_Es_ans[i],k_Es[i])
        
        #make boxes around the k's
        s_box = SurroundingRectangle(k_Es[0],buff=0.15)
        k_boxs = [Rectangle(width=s_box.width,height=0.618*s_box.width,color=GRAY_D) for i in range(9)]
        for i in range(9):
            set_new_location(k_boxs[i],k_Es[i])
            k_boxs[i].set_fill(GRAY_D,opacity=1) #box color is GRAY_D
        
        
        box_group = VGroup(*k_boxs)        
        #self.add(box_group)
        #self.add(k_Es_group) 
        
        #Arrows and labels
        ARROW_COLOR = GRAY_A
        
        even_arrows = [CurvedArrow(start_point=k_boxs[i].get_top(),end_point=k_boxs[i+2].get_top(),color=ARROW_COLOR,angle=-1.5707963267948966) for i in [0,2,4,6]]
        even_labels = [MathTex(r"\times",str(2*i+1),font_size=EQN_SIZE_3,color=ARROW_COLOR).next_to(even_arrows[i],UP) for i in range(3)]
        
        #same labels but with k-1 labeled and colored in correctly
        even_labels_colr = [MathTex(r"\times","(",str(2*i+2),"-1)",font_size=EQN_SIZE_3,color=ARROW_COLOR).next_to(even_arrows[i],UP) for i in range(3)]
        for mob in even_labels_colr:
            mob[2].color = K_COLOR #color in the k
        
        #printEnumerate(even_labels_colr[0])
        
        #self.add(*even_arrows)
        #self.add(*even_labels)
        
        odd_arrows = [CurvedArrow(start_point=k_boxs[i].get_bottom(),end_point=k_boxs[i+2].get_bottom(),color=ARROW_COLOR) for i in [1,3,5]]
        odd_labels = [MathTex(r"\times",str(2*i+2),font_size=EQN_SIZE_3,color=ARROW_COLOR).next_to(odd_arrows[i],DOWN) for i in range(3)]
        
        #self.add(*odd_arrows)
        #self.add(*odd_labels)
        
        #Add everything that was just created
        #self.add(box_group,k_Es_group,*even_arrows,*even_labels,*odd_arrows,*odd_labels)
        
        #Now animate the things out one at a time
        
        self.play(FadeIn(k_boxs[2]),Create(k_Es[2]))
        self.wait(DELAY)

        self.play(FadeIn(k_boxs[0],k_Es[0],even_arrows[0]))
        self.wait(DELAY)

        self.play(FadeIn(even_labels_colr[0]))
        self.wait(DELAY)
        self.play(TransformMatchingTex(even_labels_colr[0], even_labels[0]))
        self.wait(DELAY)
        
        Z0_temp = MathTexAndColor3(r"\mathbb{E}",r"[",r"1","]")
        Z0_temp.align_to(k_Es[0],LEFT)
        Z0_temp.align_to(k_Es[0],DOWN)
        self.play(TransformMatchingTex(k_Es[0],Z0_temp))
        self.wait(DELAY)
        
        printEnumerate(Z0_temp)
        Z0_final = Z0_temp[2].copy()
        
        self.play(FadeOut(*[Z0_temp[k] for k in [0,1,3]]))#fade out all but the "1"
        
                                   
        self.play(ReplacementTransform(Z0_final.copy(), k_Es_ans[2],path_arc=-1.57),FadeOut(k_Es[2]),Indicate(even_labels[0]))
        #self.play(FadeOut(Z0_final,target_position=k_Es[2]),FadeOut(k_Es[2]),FadeIn(k_Es_ans[2]))
        self.wait(DELAY)
                    
        self.play(FadeIn(k_boxs[4]),Create(k_Es[4]))
        self.wait(DELAY)
        
        self.play(FadeIn(even_arrows[1]))
        self.wait(DELAY)

        self.play(FadeIn(even_labels_colr[1]))
        self.wait(DELAY)
        self.play(TransformMatchingTex(even_labels_colr[1], even_labels[1]))
        self.wait(DELAY)
        
        self.play(ReplacementTransform(Z0_final.copy(), k_Es_ans[2],path_arc=-1.57),Indicate(even_labels[0]))
        self.play(ReplacementTransform(k_Es_ans[2].copy(), k_Es_ans[4],path_arc=-1.57),FadeOut(k_Es[4]),Indicate(even_labels[1]))
        
        
        
        
        
        
        self.wait(2)
        
        

0 SingleStringMathTex('\\mathbb{E}')
1 SingleStringMathTex('[')
2 SingleStringMathTex('1')
3 SingleStringMathTex(']')


In [379]:
k=2
print(r"{{k}}="+str(k))

{{k}}=2


In [582]:
%%manim -ql -v CRITICAL MoveAlongPathExample

class MoveAlongPathExample(Scene):
    def construct(self):
        d1 = Dot().set_color(ORANGE)
        l1 = Line(LEFT, RIGHT)
        l2 = VMobject()
        self.add(d1, l1,l2)
        l2.add_updater(lambda x: x.become(Line(LEFT, d1.get_center()).set_color(ORANGE)))
        self.play(MoveAlongPath(d1, l1), rate_func=linear)